In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

In [2]:
batch_size = 64

In [3]:
# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./mnist_data/',
                              train=False,
                              transform=transforms.ToTensor())

In [4]:
# Dataloader

train_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch_size,
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset,
                         batch_size=batch_size,
                         shuffle=True)

In [5]:
# Inception

class InceptionA(nn.Module):
    
    def __init__(self, in_channels):
        
        super(InceptionA, self).__init__()
        self.branch1x1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        
        self.branch5x5_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch5x5_2 = nn.Conv2d(16, 24, kernel_size=5, padding=2)
        
        self.branch3x3_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch3x3_2 = nn.Conv2d(16, 24, kernel_size=3, padding=1)
        self.branch3x3_3 = nn.Conv2d(24, 24, kernel_size=3, padding=1)
        
        self.branch_pool = nn.Conv2d(in_channels, 24, kernel_size=1)
        
    def forward(self, x):
        branch1x1 = self.branch1x1(x)
        
        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)
        
        branch3x3 = self.branch3x3_1(x)
        branch3x3 = self.branch3x3_2(branch3x3)
        branch3x3 = self.branch3x3_3(branch3x3)
        
        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)
        
        output = [branch1x1, branch5x5, branch3x3, branch_pool]
        return torch.cat(output, 1)
        

In [6]:
# CNN

class CNN_Inception(nn.Module):
    
    def __init__(self):
        
        super(CNN_Inception, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(88, 20, kernel_size=5)
        
        self.incept1 = InceptionA(in_channels=10)
        self.incept2 = InceptionA(in_channels=20)
        
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(1408, 10)
        
    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = self.incept1(x)
        x = F.relu(self.mp(self.conv2(x)))
        x = self.incept2(x)
        x = x.view(in_size, -1)
        x = self.fc(x)
        return F.log_softmax(x)

    
# intantiation

cnn_inception = CNN_Inception()

In [7]:
#optimizer

optimizer = optim.SGD(cnn_inception.parameters(), lr=0.01, momentum=0.5)

In [14]:
# train

def train(epoch):
    cnn_inception.train()
    for batch_idx, (data, labels) in enumerate(train_loader):
        data, labels = Variable(data), Variable(labels)
        
        # zero the gradient, forward, backward and running pytorch rhythm
        output = cnn_inception(data)
        optimizer.zero_grad()
        loss = F.nll_loss(output, labels)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))
        

In [15]:
# test

def test():
    cnn_inception.eval()
    test_loss = 0
    correct = 0
    for data, labels in test_loader:
        data, labels = Variable(data), Variable(labels)
        
        # output
        output = cnn_inception(data)
        
        # test loss
        test_loss += F.nll_loss(output, labels)
        
        # get the label of prediction
        pred = torch.max(output.data, 1)[1]
        correct += pred.eq(labels.data.view_as(pred)).cpu().sum()
        
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
# training the model

for epoch in range(1,10):
    train(epoch)
    test()

c:\users\shanu\anaconda3\envs\dl-udacity-pytorch\lib\site-packages\ipykernel_launcher.py:25: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
c:\users\shanu\anaconda3\envs\dl-udacity-pytorch\lib\site-packages\ipykernel_launcher.py:18: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.270877
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.242679
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.239532
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.231569
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.224316
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.127029
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.034953
Train Epoch: 1 [4480/60000 (7%)]	Loss: 1.859458
Train Epoch: 1 [5120/60000 (9%)]	Loss: 1.638036
Train Epoch: 1 [5760/60000 (10%)]	Loss: 1.278142
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.114982
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.923182
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.840831
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.611248
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.529544
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.653149
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.737344
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.624878
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.466130
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.408823
Train Epoch: 1 [12800/60000 (